In this page we simulate data from a causal model in observational setting and examine the effect of omitted variables which stochastically predict the treatment assignment.
Then we apply the tmle method for estimating the causal parameter.
We use R codes in python to run this procedure.

First import the necessary libraries:

In [1]:
import numpy as np 
import pandas as pd 
import copy
import mlp

from sklearn.linear_model import LinearRegression, LogisticRegression
from glmnet import LogitNet, ElasticNet
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import utils
from utils import simulate_x, nonlinear, simulate_params
from simulate_data import SimulateNoIntraction
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

Fix parameters for simulating data from the causal model:

In [2]:
n, p_c, p_iv, p_y, p_n = 200, 5, 5, 5, 5

True_TE = -1

In [3]:
y, A, x = SimulateNoIntraction(True_TE, n, p_c, p_iv, p_y, p_n, rho=.5, corr="AR(1)",
                                 nonlinearity_portion=.10, r1=.1, r2=1., sigma=1.)

import packages needed to run r code in python:

In [5]:
%%time
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

nr,nc = x.shape
x_r = ro.r.matrix(x, nrow=nr, ncol=nc)

CPU times: user 506 ms, sys: 670 ms, total: 1.18 s
Wall time: 271 ms


transfer data to R:

In [6]:
_ = importr("tmle")
_ = ro.r('library("tmle")')

_ = ro.r('A = c{}'.format(tuple(A.flatten())))

_ = ro.r('W = c{}'.format(tuple(x_r)))
_ = ro.r("W = matrix(W, nrow={}, ncol={})".format(nr, nc))

_ = ro.r('y = c{}'.format(tuple(y.flatten())))

Run tmle, with defaults arguments:

In [7]:
%%time
tmle_results = ro.r('results <- tmle(Y=y,A=A,W=W)')

CPU times: user 4.5 s, sys: 0 ns, total: 4.5 s
Wall time: 2.82 s


In [8]:
tmle_ests = ro.r('results$estimates')

Extract the estimate of the causal parameters, its standard error, confidence interval and the p-value:

In [9]:
TE_tmle = tmle_ests[1][0]
sd_TE_tmle = tmle_ests[1][1]
CI_TE_tmle = tmle_ests[1][2]
pvalue_TE_tmle = tmle_ests[1][3]

In [10]:
print("tmle estimate of TE")
print(TE_tmle[0])
print()
print("stderr of tmle estimate of TE")
print(np.sqrt(sd_TE_tmle)[0])
print()
print("95% confidence interval of tmle estimate of TE")
print(CI_TE_tmle)
print("p-value of tmle estimate of TE")
print(pvalue_TE_tmle)

tmle estimate of TE
-0.9174518387598916

stderr of tmle estimate of TE
0.1605504079533478

95% confidence interval of tmle estimate of TE
[1] -1.232131 -0.602773

p-value of tmle estimate of TE
[1] 1.100813e-08

